In [47]:
import pandas as pd
import numpy as np
import os
import imagesize
from sklearn.model_selection import train_test_split
from PIL import Image
from pathlib import Path
from numpy import expand_dims
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.transform import resize
from IPython.display import SVG
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model  # type: ignore
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Flatten, Dense, Activation, Dropout, Input # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore

In [2]:
image_dir = "stanford_dogs/images/Images/"
breed_list = [s for s in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir,s))]
def load_images(img_dir):
    img_arr = []
    labels = []
    for dirpath, dirnames, filenames in os.walk(img_dir):
        for filename in filenames:
            img_arr.append(np.asarray(Image.open(dirpath + "/" + filename).convert("RGB").resize((224, 224)), dtype="uint8"))
            labels.append(breed_list.index(dirpath.split("Images/")[1]))
    return np.array(img_arr), labels
img_arr, labels = load_images(image_dir)

In [49]:
def image_split(img_arr, labels, train_dist, val_dist, test_dist):
    train_imgs, valtest_imgs, train_labels, valtest_labels = train_test_split(img_arr, labels, test_size = val_dist + test_dist, train_size = train_dist, shuffle= True,stratify = labels)
    val_imgs, test_imgs, val_labels, test_labels = train_test_split(valtest_imgs, valtest_labels, test_size = val_dist/(val_dist+test_dist))
    train_labels_encoded = to_categorical(train_labels, num_classes=len(breed_list))
    val_labels_encoded = to_categorical(val_labels, num_classes=len(breed_list))
    test_labels_encoded = to_categorical(test_labels, num_classes=len(breed_list))

    return train_imgs, val_imgs, test_imgs, train_labels_encoded, val_labels_encoded, test_labels_encoded
train_imgs, val_imgs, test_imgs, train_labels, val_labels, test_labels = image_split(img_arr, labels, 0.6, 0.2, 0.2)

In [50]:
model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3), weights='resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
model.trainable = False
inputs = Input(shape=(224, 224, 3))
x = model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(len(breed_list), activation='softmax')(x)
model = Model(inputs, outputs)
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 120)            │        30,840 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,143,096 (92.10 MB)

 Trainable params: 555,384 (2.12 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [52]:
history = model.fit(train_imgs, np.array(train_labels), epochs=10, batch_size=32, verbose=1, shuffle=False, validation_data=(val_imgs, val_labels))

Epoch 1/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 3405s 9s/step - accuracy: 0.2748 - loss: 2.6979 - val_accuracy: 0.5224 - val_loss: 1.8128
Epoch 2/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1701s 4s/step - accuracy: 0.3633 - loss: 2.2847 - val_accuracy: 0.5447 - val_loss: 1.6527
Epoch 3/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1701s 4s/step - accuracy: 0.4040 - loss: 2.0997 - val_accuracy: 0.5617 - val_loss: 1.5546
Epoch 4/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1759s 5s/step - accuracy: 0.4213 - loss: 2.0049 - val_accuracy: 0.5797 - val_loss: 1.4927
Epoch 5/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 1813s 5s/step - accuracy: 0.4410 - loss: 1.8871 - val_accuracy: 0.5828 - val_loss: 1.4598
Epoch 6/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 2051s 5s/step - accuracy: 0.4540 - loss: 1.8428 - val_accuracy: 0.5790 - val_loss: 1.4589
Epoch 7/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 3362s 9s/step - accuracy: 0.4712 - loss: 1.7706 - val_accuracy: 0.5724 - val_loss: 1.4775
Epoch 8/10
386/386 ━━━━━━━━━━━━━━━━━━━━ 2049s 5s/step - accuracy: 0.4892 - loss: 1.6999 - 